In [1]:
import scala.util.Try
import scala.math.max
import org.apache.spark._
import org.apache.log4j._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.sql.functions.{concat, lit}
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.functions.expr
import org.apache.spark.sql.functions._
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.sql.types._
import java.net.URLDecoder

In [2]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{DenseVector, SparseVector, Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}

In [3]:
val spark = SparkSession.builder().appName("Victoria").config("spark.master", "yarn").getOrCreate()

spark = org.apache.spark.sql.SparkSession@15f84cb6


org.apache.spark.sql.SparkSession@15f84cb6

In [4]:
val train = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .option("encoding","utf-8")
                    .csv("/labs/slaba05/lab05_train.csv")
val test = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .option("encoding","utf-8")
                    .csv("/labs/slaba05/lab05_test.csv")

train = [_c0: string, ID: string ... 115 more fields]
test = [_c0: string, ID: string ... 114 more fields]


[_c0: string, ID: string ... 114 more fields]

In [5]:
train.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- CR_PROD_CNT_IL: string (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: string (nullable = true)
 |-- APP_REGISTR_RGN_CODE: string (nullable = true)
 |-- PRC_ACCEPTS_A_POS: string (nullable = true)
 |-- PRC_ACCEPTS_A_TK: string (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: string (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: string (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- CNT_ACCEPTS_TK: string (nullable = true)
 |--

In [5]:
var df_train = train.drop("APP_CAR", "APP_COMP_TYPE", "APP_COMP_TYPE", "APP_DRIVING_LICENSE", "APP_EMP_TYPE", 
                      "APP_KIND_OF_PROP_HABITATION", "APP_MARITAL_STATUS", "APP_POSITION_TYPE", "APP_TRAVEL_PASS",
                      "APP_REGISTR_RGN_CODE", "APP_TRAVEL_PASS", 
                      "CLNT_JOB_POSITION_TYPE", "PACK", "_c0",
                     "CLNT_TRUST_RELATION", "CLNT_JOB_POSITION")
var df_test = test.drop("APP_CAR", "APP_COMP_TYPE", "APP_COMP_TYPE", "APP_DRIVING_LICENSE", "APP_EMP_TYPE", 
                      "APP_KIND_OF_PROP_HABITATION", "APP_MARITAL_STATUS", "APP_POSITION_TYPE", "APP_TRAVEL_PASS",
                      "APP_REGISTR_RGN_CODE", "APP_TRAVEL_PASS", 
                      "CLNT_JOB_POSITION_TYPE", "PACK", "_c0",
                     "CLNT_TRUST_RELATION", "CLNT_JOB_POSITION")

df_train = [ID: string, CR_PROD_CNT_IL: string ... 101 more fields]
df_test = [ID: string, CR_PROD_CNT_IL: string ... 100 more fields]


[ID: string, CR_PROD_CNT_IL: string ... 100 more fields]

In [6]:
var feature_list = Array[String]()
for (t <- df_train.columns if (t!="ID"))
{ df_train = df_train.withColumn(t, col(t).cast(FloatType))
        feature_list +:= t}
val b = feature_list.filter(! _.contains("TARGET"))

feature_list = Array(TARGET, LDEAL_ACT_DAYS_PCT_CURR, LDEAL_ACT_DAYS_PCT_TR4, LDEAL_ACT_DAYS_PCT_TR, TURNOVER_DYNAMIC_CC_3M, LDEAL_DELINQ_PER_MAXYWZ, LDEAL_ACT_DAYS_PCT_AAVG, LDEAL_ACT_DAYS_PCT_TR3, MED_DEBT_PRC_YWZ, REST_DYNAMIC_CC_3M, LDEAL_ACT_DAYS_ACC_PCT_AVG, AVG_PCT_DEBT_TO_DEAL_AMT, TURNOVER_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YWZ, REST_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YQZ, TRANS_CNT_TENDENCY3M, MED_DEBT_PRC_YQZ, TRANS_AMOUNT_TENDENCY3M, CLNT_SALARY_VALUE, TURNOVER_DYNAMIC_PAYM_1M, LDEAL_DELINQ_PER_MAXYQZ, TURNOVER_DYNAMIC_PAYM_3M, DEAL_GRACE_DAYS_ACC_MAX, CLNT_SETUP_TENOR, LDEAL_YQZ_PC, MAX_PCLOSE_DATE, TURNOVER_DYNAMIC_CUR_3M, DEAL_GRACE_DAYS_ACC_AVG, CNT_TRAN_SUP_TENDENCY1M, REST_DYNAMIC_PAYM_1M, SUM_TRAN_ATM_TENDENCY1M, DEAL_YWZ_IR_MAX, SUM_TRAN_SUP_TENDENCY1M, ...


Array(TARGET, LDEAL_ACT_DAYS_PCT_CURR, LDEAL_ACT_DAYS_PCT_TR4, LDEAL_ACT_DAYS_PCT_TR, TURNOVER_DYNAMIC_CC_3M, LDEAL_DELINQ_PER_MAXYWZ, LDEAL_ACT_DAYS_PCT_AAVG, LDEAL_ACT_DAYS_PCT_TR3, MED_DEBT_PRC_YWZ, REST_DYNAMIC_CC_3M, LDEAL_ACT_DAYS_ACC_PCT_AVG, AVG_PCT_DEBT_TO_DEAL_AMT, TURNOVER_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YWZ, REST_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YQZ, TRANS_CNT_TENDENCY3M, MED_DEBT_PRC_YQZ, TRANS_AMOUNT_TENDENCY3M, CLNT_SALARY_VALUE, TURNOVER_DYNAMIC_PAYM_1M, LDEAL_DELINQ_PER_MAXYQZ, TURNOVER_DYNAMIC_PAYM_3M, DEAL_GRACE_DAYS_ACC_MAX, CLNT_SETUP_TENOR, LDEAL_YQZ_PC, MAX_PCLOSE_DATE, TURNOVER_DYNAMIC_CUR_3M, DEAL_GRACE_DAYS_ACC_AVG, CNT_TRAN_SUP_TENDENCY1M, REST_DYNAMIC_PAYM_1M, SUM_TRAN_ATM_TENDENCY1M, DEAL_YWZ_IR_MAX, SUM_TRAN_SUP_TENDENCY1M, ...

In [7]:
for (t <- df_test.columns if (t!="ID"))
{ df_test = df_test.withColumn(t, col(t).cast(FloatType))}

In [8]:
df_test = df_test.na.fill(value=0.0)
df_train = df_train.na.fill(value=0.0)

df_test = [ID: string, CR_PROD_CNT_IL: float ... 100 more fields]
df_train = [ID: string, CR_PROD_CNT_IL: float ... 101 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 101 more fields]

In [9]:
val assembler = new VectorAssembler()
                    .setInputCols(b)
                    .setOutputCol("features")

assembler = vecAssembler_4c537c708dc2


vecAssembler_4c537c708dc2

In [10]:
df_train = df_train.withColumn("TARGET", col("TARGET").cast(IntegerType))

df_train = [ID: string, CR_PROD_CNT_IL: float ... 101 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 101 more fields]

In [11]:
val train_data=assembler.transform(df_train).repartition(30)

train_data = [ID: string, CR_PROD_CNT_IL: float ... 102 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 102 more fields]

In [12]:
val evaluator = new BinaryClassificationEvaluator()
                        .setLabelCol("TARGET")
                        .setRawPredictionCol("probability")
                        .setMetricName("areaUnderROC")

evaluator = binEval_37563a2cb2ca


binEval_37563a2cb2ca

In [13]:
val rf = new RandomForestClassifier()
      .setFeaturesCol("features")
      .setLabelCol("TARGET")
      .setNumTrees(50)
      .setMaxDepth(10)
      .setSubsamplingRate(0.7)

rf = rfc_9ab63b3f4377


rfc_9ab63b3f4377

In [14]:
val test_data=assembler.transform(df_test).repartition(30)

test_data = [ID: string, CR_PROD_CNT_IL: float ... 101 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 101 more fields]

In [15]:
val model_rf = rf.fit(train_data)

model_rf = RandomForestClassificationModel (uid=rfc_9ab63b3f4377) with 50 trees


RandomForestClassificationModel (uid=rfc_9ab63b3f4377) with 50 trees

In [16]:
val predictions_train = model_rf.transform(train_data)

predictions_train = [ID: string, CR_PROD_CNT_IL: float ... 105 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 105 more fields]

In [17]:
val accuracy = evaluator.evaluate(predictions_train)

accuracy = 0.8311066846854087


0.8311066846854087

In [18]:
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.16889331531459129


In [19]:
val predictions_test = model_rf.transform(test_data)

predictions_test = [ID: string, CR_PROD_CNT_IL: float ... 104 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 104 more fields]

In [20]:
val sparse_to_dense = udf((v: Vector) =>  v.toArray)

sparse_to_dense = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [21]:
predictions_test.printSchema()

root
 |-- ID: string (nullable = true)
 |-- CR_PROD_CNT_IL: float (nullable = false)
 |-- AMOUNT_RUB_CLO_PRC: float (nullable = false)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: float (nullable = false)
 |-- PRC_ACCEPTS_A_POS: float (nullable = false)
 |-- PRC_ACCEPTS_A_TK: float (nullable = false)
 |-- TURNOVER_DYNAMIC_IL_1M: float (nullable = false)
 |-- CNT_TRAN_AUT_TENDENCY1M: float (nullable = false)
 |-- SUM_TRAN_AUT_TENDENCY1M: float (nullable = false)
 |-- AMOUNT_RUB_SUP_PRC: float (nullable = false)
 |-- PRC_ACCEPTS_A_AMOBILE: float (nullable = false)
 |-- SUM_TRAN_AUT_TENDENCY3M: float (nullable = false)
 |-- PRC_ACCEPTS_TK: float (nullable = false)
 |-- PRC_ACCEPTS_A_MTP: float (nullable = false)
 |-- REST_DYNAMIC_FDEP_1M: float (nullable = false)
 |-- CNT_TRAN_AUT_TENDENCY3M: float (nullable = false)
 |-- CNT_ACCEPTS_TK: float (nullable = false)
 |-- REST_DYNAMIC_SAVE_3M: float (nullable = false)
 |-- CR_PROD_CNT_VCU: float (nullable = false)
 |-- REST_AVG_CUR: float (nullable = false)

In [22]:
val to_file = predictions_test.select(col("ID").alias("id"), sparse_to_dense(col("probability")).getItem(1).alias("target")).coalesce(1)

to_file = [id: string, target: double]


[id: string, target: double]

In [23]:
to_file.show()

+------+--------------------+
|    id|              target|
+------+--------------------+
|338743| 0.03496024263565974|
|396274|0.026194646313301866|
|317805| 0.03382502865828622|
|398424|0.019947511921613183|
|373496|  0.1538554613847935|
|209206| 0.24028680539064584|
|319688|  0.0437684282443641|
|345326| 0.04620195805159554|
|373023|0.015003051249999524|
|225004| 0.11518426275774575|
|192599| 0.10596093615330558|
|470522|   0.090876727493904|
|537732| 0.07387945741189966|
|444496|0.030672659283466453|
|244935|0.058293322431370934|
|485741| 0.12603929645228734|
|551227|  0.0859178997921326|
|395048| 0.15254750674652762|
|255069|0.054553880189411895|
|274320|  0.1821389510951845|
+------+--------------------+
only showing top 20 rows



In [24]:
to_file.write.mode("overwrite").format("csv").option("delimiter", "\t").option("header", "True").option("encoding", "UTF-8").save("lab05")

In [28]:
import sys.process._
"""hdfs dfs -ls lab05""".!!

lastException: Throwable = null
"Found 2 items
-rw-r--r--   3 tatiana.gorlova tatiana.gorlova          0 2022-11-07 22:56 lab05/_SUCCESS
-rw-r--r--   3 tatiana.gorlova tatiana.gorlova    1200796 2022-11-07 22:56 lab05/part-00000-d8baf47d-f313-46a5-abbf-4d630f685beb-c000.csv
"


In [29]:
"""mv 5/part-00000-d8baf47d-f313-46a5-abbf-4d630f685beb-c000.csv lab05.csv""".!!

res55: String = ""
